# Verify S3_BUCKET Bucket Creation

In [ ]:
import boto3
import sagemaker
import time

session = boto3.session.Session()
region = session.region_name

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

account_id = boto3.client("sts").get_caller_identity().get("Account")

s3 = boto3.Session().client(service_name="s3", region_name=region)

airflow_env_name = "mwaa-" + region + "-" + account_id
airflow_vpc_name = "mwaa-vpc" + region + "-" + account_id

In [ ]:
setup_s3_bucket_passed = False
%store -r airflow_bucket_name
%store airflow_env_name
%store airflow_vpc_name

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=airflow_bucket_name)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(airflow_bucket_name, response, e))

In [ ]:
%store setup_s3_bucket_passed

# Configure Airflow DAG Files before uploading to S3 Bucket

In [ ]:
with open("./dags/config.py", "r") as f:
    lines = f.readlines()

with open("./dags/config.py", "w") as f:
    for line in lines:
        line = line.replace("{0}", region)
        line = line.replace("{1}", bucket)
        f.write(line)

# Copy Airflow DAG to S3 Bucket

In [ ]:
%store -r s3_mwaa_private_path
%store -r s3_mwaa_dags_private_path
%store -r s3_mwaa_pipeline_private_path

In [ ]:
!aws s3 cp --recursive ./dags/pipeline $s3_mwaa_pipeline_private_path
!aws s3 cp ./dags/config.py $s3_mwaa_dags_private_path/config.py
!aws s3 cp ./dags/bert_reviews.py $s3_mwaa_dags_private_path/bert_reviews.py
!aws s3 cp ./dags/requirements.txt $s3_mwaa_private_path/requirements.txt

In [ ]:
!aws s3 ls $s3_mwaa_private_path
!aws s3 ls $s3_mwaa_dags_private_path

# Configure Temporary IAM Role Policy for MWAA VPC

In [ ]:
# Check number of policies attached to TeamRole, we need to have nine
iam = boto3.resource("iam")
iam_client = boto3.client("iam")
team_role_arn = iam.Role("TeamRole").arn

team_role = iam.Role("TeamRole")

aws_managed_policies = [p for p in team_role.attached_policies.all()]

if len(aws_managed_policies) >= 10:
    print(
        "You have: {} policies attached to TeamRole, you need downsize to 9 Policies so that we can add an MWAA VPC Creation Policy.".format(
            len(aws_managed_policies)
        )
    )
    print("Please do NOT continue unless until you run this and get a Success message")
else:
    print("Success! Please Continue...")

In [ ]:
mwaa_vpc_policy_json = open("./src/mwaa_vpc_policy.json", "r").read()
mwaa_vpc_policy_json = mwaa_vpc_policy_json.replace("{0}", region)
mwaa_vpc_policy_json = mwaa_vpc_policy_json.replace("{1}", account_id)

In [ ]:
%store team_role_arn

In [ ]:
response = iam_client.create_policy(PolicyName="mwaa_vpc_policy", PolicyDocument=mwaa_vpc_policy_json)

mwaa_vpc_policy_arn = response["Policy"]["Arn"]

# Create VPC MWAA Environment - Please be patient this can take around 10 Minutes.

In [ ]:
cloudformation = boto3.resource("cloudformation")

mwaa_vpc_template_yaml = open("./cfn/mwaa_vpc_template.yaml", "r").read()

response = cloudformation.create_stack(
    StackName="mwaa-vpc-stack",
    TemplateBody=mwaa_vpc_template_yaml,
    Parameters=[
        {"ParameterKey": "EnvironmentName", "ParameterValue": airflow_vpc_name},
    ],
    ResourceTypes=[
        "AWS::EC2::VPC",
    ],
    OnFailure="ROLLBACK",
    EnableTerminationProtection=False,
)

stack_status = "IN_PROGRESS"

print("Starting deployment of VPC {}. \n".format(airflow_vpc_name))

while stack_status != "CREATE_COMPLETE":
    stack_status = cloudformation.Stack("mwaa-vpc-stack").stack_status
    time.sleep(30)
    print("Still waiting....")

print("\n Sucess! VPC {} has been deployed sucessfully.".format(airflow_vpc_name))

In [ ]:
vpc_outputs = cloudformation.Stack("mwaa-vpc-stack").outputs

airflow_sg_id = None
for output in vpc_outputs:
    if output["OutputKey"] == "IngressSecurityGroup":
        airflow_sg_id = output["OutputValue"]
        break

subnet_index_list = ["PrivateSubnet1", "PrivateSubnet2"]
airflow_subnet_ids = []

for output in vpc_outputs:
    for subnet_index in subnet_index_list:
        if output["OutputKey"] == subnet_index:
            airflow_subnet_ids.append(output["OutputValue"])

In [ ]:
%store airflow_sg_id
%store airflow_subnet_ids

# Configure IAM Role Policy for MWAA

In [ ]:
# Remove MWAA VPC Policy only needed for VPC Creation
response = iam_client.detach_role_policy(RoleName="TeamRole", PolicyArn=mwaa_vpc_policy_arn)

In [ ]:
# Check number of policies attached to TeamRole, we need to have nine
team_role = iam.Role("TeamRole")

aws_managed_policies = [p for p in team_role.attached_policies.all()]

if len(aws_managed_policies) >= 10:
    print(
        "You have: {0} policies attached to TeamRole, you need downsize to 9 Policies so that we can add an MWAA Policy.".format(
            len(aws_managed_policies)
        )
    )
    print("Please do NOT continue unless until you run this and get a Success message")
else:
    print("Success! Please Continue...")

In [ ]:
mwaa_policy_json = open("./src/mwaa_policy.json", "r").read()
mwaa_policy_json = mwaa_policy_json.replace("{0}", region)
mwaa_policy_json = mwaa_policy_json.replace("{1}", account_id)
mwaa_policy_json = mwaa_policy_json.replace("{2}", airflow_env_name)
mwaa_policy_json = mwaa_policy_json.replace("{3}", airflow_bucket_name)

mwaa_assume_policy_json = open("./src/mwaa_assume_policy.json", "r").read()

In [ ]:
response = iam_client.create_policy(PolicyName="mwaa_policy", PolicyDocument=mwaa_policy_json)

response = iam_client.attach_role_policy(RoleName="TeamRole", PolicyArn=response["Policy"]["Arn"])

response = iam_client.update_assume_role_policy(RoleName="TeamRole", PolicyDocument=mwaa_assume_policy_json)

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}